In [18]:
import numpy as np
import pandas as pd

## constant
gravity_acceleration = 9.81  # m/s2
air_density = 1.1985 # kg/m3
rolling_resist_coeff_1 = 0.008 # constant
rolling_resist_coeff_2 = 0.00012  # per m/s

## define EV attributes ###

EV_100mile_attr = {"type": "BEV_100mile",
                    "vehicle_weight": 1659, #kg
                   "drag_coeff":0.315,
                   "frontal_area":2.755,
                   "battery_size_kwh":30.4128} # per m/s

EV_300mile_attr = {"type": "BEV_300mile",
                    "vehicle_weight": 2270, #kg
                   "drag_coeff":0.3,
                   "frontal_area":2.832,
                   "battery_size_kwh":101.177} # per m/s

FCEV_attr = {"type": "FCEV",
                    "vehicle_weight": 1760, #kg
                   "drag_coeff":0.3,
                   "frontal_area":2.786,
                   "battery_size_kwh":1.823472} # per m/s

PS_PHEV_attr = {"type": "PS_PHEV",
                    "vehicle_weight": 1712, #kg
                   "drag_coeff":0.311,
                   "frontal_area":2.372,
                   "battery_size_kwh":8.1147744
                   } # per m/s


SER_PHEV_attr = {"type": "SER_PHEV",
                    "vehicle_weight": 1893, #kg
                   "drag_coeff":0.3,
                   "frontal_area":2.565,
                   "battery_size_kwh":14.888772
                   } # per m/s

PAR_HEV_attr = {"type": "PAR_HEV",
                    "vehicle_weight": 1639.7, #kg
                   "drag_coeff":0.3,
                   "frontal_area":2.25,
                   "battery_size_kwh":1.458
                   } # per m
 

PS_PHEV_eng_on_param = [-2.67677801, 4.24002192e-02, 9.15611588e-05, 1.65119099e-04, -1.93716152]
PS_PHEV_mot1_pos_param = [-0.11713582, 0.00028122, 0.04196995]

SER_PHEV_eng_on_param = [-3.06319594, 8.08387735e-02, 3.75583185e-05, 1.66369333e-05, -2.27318062]
SER_PHEV_mot1_pos_param = [0.0, 0.00024091, 0.04576871]

PAR_HEV_eng_on_param = [-2.67418185,  1.54580705,  1.81889166,  9.46418049e-05, 0.69051686]

In [12]:
## define EV class for store EV attributes ##
class electric_vehicle:
    def __init__(self,vid,veh):
        self.id = vid
        self.type = veh['type']
        self.weight = veh['vehicle_weight']
        self.drag_coeff = veh['drag_coeff']
        self.frontal_area = veh['frontal_area']
        self.battery_size = veh['battery_size_kwh']
        
## basic calculation##
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

def update_soc(initial_soc, energy_consumption, battery_size):
    energy_available = initial_soc * battery_size
    energy_remaining = energy_available - energy_consumption
    latest_soc = energy_remaining / battery_size
    return latest_soc

## power demand calculation ##
def vehicle_power_calculator(speed, acc, grade, weight, frontal_area, drag_coeff):
    rolling_resistance_coeff = rolling_resist_coeff_1 + speed * rolling_resist_coeff_2
    rolling_resistance = rolling_resistance_coeff * weight * gravity_acceleration * np.cos(grade)
    aerodynamic_resistance = 0.5 * air_density * frontal_area * drag_coeff * (speed ** 2)
    uphill_drag = weight * gravity_acceleration * np.sin(grade)
    acceleration_load = weight * acc
    vehicle_power_demand_watt = (acceleration_load + aerodynamic_resistance + rolling_resistance + uphill_drag) * speed
    return vehicle_power_demand_watt

## vehicle control (HEV-only) calculation ##
def eng_on_fitting(SOC, Speed, max_VSP, min_VSP, eng_on_param, target_soc=0.3):
    x = [(SOC>target_soc), Speed, max_VSP, min_VSP, 1]
    eng_on_score = np.dot(eng_on_param, x)
    eng_on_prob = sigmoid(eng_on_score)
    return eng_on_prob

def mot1_pos_fitting(Speed, VSP, mot1_pos_param):
    x = [Speed, VSP, 1]
    mot_pos_score = np.dot(mot1_pos_param, x)
    mot_pos_prob = sigmoid(mot_pos_score)
    return mot_pos_prob

In [13]:
## BEV energy use calculation ##
def ALLEV_energy_calculator(veh_type, vsp):
    x_elec = [vsp * (vsp >= 0), (vsp >= 0), vsp * (vsp < 0), (vsp < 0)]
    if veh_type == "BEV_100mile":
        x_elec_coeff = [1.9466, 930.2749, 1.3168, 765.0286]
        elec_consumption = np.dot(x_elec_coeff, x_elec)
    elif veh_type == "BEV_300mile":
        x_elec_coeff = [2.6535, 687.9322, 1.7740, 601.6535]
        elec_consumption = np.dot(x_elec_coeff, x_elec)
    return elec_consumption

In [14]:
## FCEV energy use calculation ##
def FCEV_energy_calculator(vsp, initial_soc, time_step):
    x_fuel = [(vsp < 0.0), initial_soc * (vsp >= 0.0), vsp * (vsp >= 0.0), (vsp >= 0.0)]
    x_fuel_coeff = [3892.0, -1.28898e+05, 4.0256, 97903.299]
    fuel_consumption = np.dot(x_fuel_coeff, x_fuel)
    x_elec = [initial_soc * (vsp < 0.0), vsp * (vsp < 0.0), fuel_consumption * time_step * (vsp < 0.0), (vsp < 0.0), 
    initial_soc * (vsp >= 0.0), vsp * (vsp >= 0.0), fuel_consumption * time_step * (vsp >= 0.0), (vsp >= 0.0)]
    x_elec_coeff = [3.8071e+03, 0.79744, -3.0205, -3029.3013, 1.07364e+04, 1.4756, -3.7569, -4848.2654]
    elec_consumption = np.dot(x_elec_coeff, x_elec)
    return pd.Series((fuel_consumption, elec_consumption))

In [15]:
## Parallel hybrid energy use calculation ##
def eng_on_fitting_parallel(Speed, VSP, SOC, eng_on_param, target_soc=0.45):
    x = [(SOC>target_soc+0.05), (SOC<=target_soc+0.05) & (SOC>target_soc), (SOC<=target_soc), VSP, 1]
    eng_on_score = np.dot(eng_on_param, x)
    eng_on_prob = sigmoid(eng_on_score)
    return eng_on_prob

def mot1_pos_fitting_parallel(Speed, VSP, SOC):
    x_mot_pos_prob = [(SOC < 0.45) * (Speed < 3.7) * (VSP < 0.0),
    (SOC < 0.45) * (Speed < 3.7) * (VSP >= 0.0),
    (SOC < 0.45) * (Speed >= 3.7) * (VSP < 230.0),
    (SOC < 0.45) * (Speed >= 3.7) * (VSP >= 230.0),
    (SOC >= 0.45) * (Speed < 3.7) * (VSP < -100),
    (SOC >= 0.45) * (Speed >= 3.7) * (VSP < -100),
    (SOC >= 0.45) * (VSP < 0) * (VSP >= -100),
    (SOC >= 0.45) * (VSP >= 0)]
    mot_pos_prob_coeff  = [0.699, 0.371, 0.097, 0.018, 0.925, 0.038, 0.668, 0.992]
    mot_pos_prob = np.dot(mot_pos_prob_coeff, x_mot_pos_prob)
    return mot_pos_prob

def PAR_HEV_energy_calculator(SOC, Speed, VSP, eng_on_param):
    eng_on_prob = eng_on_fitting_parallel(Speed, VSP, SOC, eng_on_param)
    mot1_pos_prob = mot1_pos_fitting_parallel(Speed, VSP, SOC)
    mot1_pos_eng_on_prob = eng_on_prob * mot1_pos_prob
    mot1_neg_eng_on_prob = eng_on_prob * (1 - mot1_pos_prob)
    fuel_eng_off = 0.0
    x_elec_eng_off = [(VSP>=0) * VSP, (VSP>=0), (VSP<0)*VSP, (VSP<0)]
    x_elec_eng_off_coeff = [1.962524, 570.9734794284032, 1.2682302, 550.8562089109209]
    elec_eng_off = np.dot(x_elec_eng_off_coeff, x_elec_eng_off)
    x_fuel_eng_on_mot1_pos = [SOC, VSP, 1]
    x_fuel_eng_on_mot1_pos_coeff = [-4.20632612e+04, 3.97007485, 25795.593398371613]
    fuel_eng_on_mot1_pos = np.dot(x_fuel_eng_on_mot1_pos_coeff, x_fuel_eng_on_mot1_pos)
    x_elec_eng_on_mot1_pos = [SOC * (SOC>=0.45), VSP * (SOC>=0.45), (SOC>=0.45), VSP * (SOC<0.45), (SOC<0.45)]
    x_elec_eng_on_mot1_pos_coeff = [2.97321643e+04, 2.32793218e-01, -12204.642108379012, 0.22400676, 484.2424644001493]
    elec_eng_on_mot1_pos = np.dot(x_elec_eng_on_mot1_pos_coeff, x_elec_eng_on_mot1_pos)
    x_fuel_eng_on_mot1_neg = [(VSP<0.0), SOC * (VSP>=0.0), VSP * (VSP>=0.0), (VSP>=0.0)]
    x_fuel_eng_on_mot1_neg_coeff = [805.2133096730364, -2.78938755e+05, 4.36362293, 140439.00530550053]
    fuel_eng_on_mot1_neg = np.dot(x_fuel_eng_on_mot1_neg, x_fuel_eng_on_mot1_neg_coeff)
    x_elec_eng_on_mot1_neg = [SOC * (VSP<0.0), VSP * (VSP<0.0), (VSP<0.0), SOC * (VSP>=0.0), VSP * (VSP>=0.0), (VSP>=0.0)]
    x_elec_eng_on_mot1_neg_coeff = [2.32002319e+03, 1.32649525, -385.50192025424167, 8.56073873e+04, 2.76506566e-03, -38815.57631950699]
    elec_eng_on_mot1_neg = np.dot(x_elec_eng_on_mot1_neg_coeff, x_elec_eng_on_mot1_neg)
    fuel_consumption = fuel_eng_off * (1 - eng_on_prob) + fuel_eng_on_mot1_pos * mot1_pos_eng_on_prob + fuel_eng_on_mot1_neg * mot1_neg_eng_on_prob
    elec_consumption = elec_eng_off * (1 - eng_on_prob) + elec_eng_on_mot1_pos * mot1_pos_eng_on_prob + elec_eng_on_mot1_neg * mot1_neg_eng_on_prob
    return pd.Series((fuel_consumption, elec_consumption))

In [16]:
## Series hybrid energy use calculation ##
def SER_PHEV_energy_calculator(SOC, Speed, VSP, max_VSP, min_VSP, eng_on_param, mot1_pos_param):
    eng_on_prob = eng_on_fitting(SOC, Speed, max_VSP, min_VSP, eng_on_param)
    mot1_pos_prob = mot1_pos_fitting(Speed, VSP, mot1_pos_param)
    mot1_pos_eng_on_prob = eng_on_prob * mot1_pos_prob
    mot1_neg_eng_on_prob = eng_on_prob * (1 - mot1_pos_prob)
    fuel_eng_off = 0.0
    x_elec_eng_off = [(VSP>=0) * VSP, (VSP>=0), (VSP<0)*VSP, (VSP<0)]
    x_elec_eng_off_coeff = [2.28165196, 476.1935775980219, 1.56366896, 538.2556373432371]
    elec_eng_off = np.dot(x_elec_eng_off_coeff, x_elec_eng_off)
    x_eng_on_mot1_pos = [(SOC>=0.36), SOC * (SOC<0.36), VSP, 1]
    x_fuel_eng_on_mot1_pos_coeff = [-2.74509128e+05, -9.09039559e+05,  4.75220283, 288932.8393509499]
    x_elec_eng_on_mot1_pos_coeff = [7.80310767e+04, 2.57519913e+05, 1.39387618e-01, -78908.847443407]
    fuel_eng_on_mot1_pos = np.dot(x_fuel_eng_on_mot1_pos_coeff, x_eng_on_mot1_pos)
    elec_eng_on_mot1_pos = np.dot(x_elec_eng_on_mot1_pos_coeff, x_eng_on_mot1_pos)
    x_eng_on_mot1_neg =  [SOC * (SOC<0.3), (SOC>=0.3),  VSP, 1]
    x_fuel_eng_on_mot1_neg_coeff = [-1.18233539e+06, -3.54980210e+05,  3.90625854, 385522.6214090834]
    x_elec_eng_on_mot1_neg_coeff = [ 2.25179949e+05,  6.75313253e+04, -1.79147887e-02, -75129.8799274783]
    fuel_eng_on_mot1_neg = np.dot(x_fuel_eng_on_mot1_neg_coeff, x_eng_on_mot1_neg)
    elec_eng_on_mot1_neg = np.dot(x_elec_eng_on_mot1_neg_coeff, x_eng_on_mot1_neg)
    fuel_consumption = fuel_eng_off * (1 - eng_on_prob) + fuel_eng_on_mot1_pos * mot1_pos_eng_on_prob + fuel_eng_on_mot1_neg * mot1_neg_eng_on_prob
    elec_consumption = elec_eng_off * (1 - eng_on_prob) + elec_eng_on_mot1_pos * mot1_pos_eng_on_prob + elec_eng_on_mot1_neg * mot1_neg_eng_on_prob
    return pd.Series((fuel_consumption, elec_consumption))

In [19]:
## Power-split hybrid energy use calculation ##
def PS_PHEV_energy_calculator(SOC, Speed, VSP, max_VSP, min_VSP, eng_on_param, mot1_pos_param):
    eng_on_prob = eng_on_fitting(SOC, Speed, max_VSP, min_VSP, eng_on_param)
    mot1_pos_prob = mot1_pos_fitting(Speed, VSP, mot1_pos_param)
    mot1_pos_eng_on_prob = eng_on_prob * mot1_pos_prob
    mot1_neg_eng_on_prob = eng_on_prob * (1 - mot1_pos_prob)
    fuel_eng_off = 0.0
    x_elec_eng_off = [(VSP>=0) * VSP, (VSP>=0), (VSP<0)*VSP, (VSP<0)]
    x_elec_eng_off_coeff = [2.45157107, 144.38601445994937, 1.33505183, 346.41790648524875]
    elec_eng_off = np.dot(x_elec_eng_off_coeff, x_elec_eng_off)
    x_eng_on_mot1_pos = [(SOC>=0.36), SOC * (SOC<0.36), VSP, 1]
    x_fuel_eng_on_mot1_pos_coeff = [-3.38597796e+05, -1.16994098e+06, 3.3584077, 363839.0015780216]
    x_elec_eng_on_mot1_pos_coeff = [1.69653646e+05, 5.79011468e+05, 5.57667998e-01, -173651.62228421587]
    fuel_eng_on_mot1_pos = np.dot(x_fuel_eng_on_mot1_pos_coeff, x_eng_on_mot1_pos)
    elec_eng_on_mot1_pos = np.dot(x_elec_eng_on_mot1_pos_coeff, x_eng_on_mot1_pos)
    x_fuel_eng_on_mot1_neg = [SOC * (SOC<0.3), SOC * (SOC<0.36) * (SOC>=0.3), (SOC>=0.36), VSP, 1]
    x_fuel_eng_on_mot1_neg_coeff = [-2.03315356e+05, -2.76856773e+05, -7.16715346e+04, 4.14674728, 87600.84724097612]
    fuel_eng_on_mot1_neg = np.dot(x_fuel_eng_on_mot1_neg_coeff, x_fuel_eng_on_mot1_neg)
    x_elec_eng_on_mot1_neg = [Speed, SOC * (SOC<0.3), SOC * (SOC<0.36) * (SOC>=0.3), (SOC>=0.36), VSP, 1]
    x_elec_eng_on_mot1_neg_coeff = [3.47681620e+02, -6.92437852e+04, -3.36729340e+04, -1.46818108e+04, -3.51383515e-01, 3199.503220777504]
    elec_eng_on_mot1_neg = np.dot(x_elec_eng_on_mot1_neg_coeff, x_elec_eng_on_mot1_neg)
    fuel_consumption = fuel_eng_off * (1 - eng_on_prob) + fuel_eng_on_mot1_pos * mot1_pos_eng_on_prob + fuel_eng_on_mot1_neg * mot1_neg_eng_on_prob
    elec_consumption = elec_eng_off * (1 - eng_on_prob) + elec_eng_on_mot1_pos * mot1_pos_eng_on_prob + elec_eng_on_mot1_neg * mot1_neg_eng_on_prob
    return pd.Series((fuel_consumption, elec_consumption))